# 2. Finding Advertising Keywards on Web Pages

# * Brief History of online advertising

* 1978년 5월 - 첫 번째 마케팅 스팸 메시지....DEC Marketing representation
- 관련 내용은 아래 사이트에 들어가면 reaction 까지 나옴
http://www.templetons.com/brad/spamreact.html
* 1994년에는 AT&T에서 첫 번째 배너를 만듬 with Hotwired.com

<img src = im1.png>

* 1996년에는 첫 번째 인터렉티브 광고 HP

<img src = im2.png width = 600>

* 여러 종류의 인터넷 광고들...
- Search Advertising
- Display Advertising
- E-mail Advertising
- Classifieds
- Sponsorships
…

# 우리가 하려는 일은?
* 웹 페이지 context에 맞는 적절한 광고를 찾아서 웹 페이지에 띄우고 이를 통해 웹 페이지는 페이지에 적절한 광고가 떠서 돈을 벌고 우리는 광고 CTR을 높여서 수익률을 극대화 하자

* 그러기 위해선....이 웹페이지의 context를 알아야 한다.
* 다음에 나오는 내용들은 웹 페이지의 context를 알기 위한 방법들

# 2.1 Keyword Extraction as a Classification Task

* 웹 페이지에서 광고 키워드를 찾아내는 것은 광고 키워드와 비 광고 키워드로 classification하는 일에서 부터 시작됨
* 이걸 위해서는 우선 웹 페이지의 텍스트를 tokenizing하고, 문장에서 빠짐없이 단어와 구로 나누는 일이 필요함

# Comparing and searching strings - Using ngram

* pip install ngram

* 두 개의 스트링 비교문 compare():

In [2]:
import ngram

In [4]:
ngram.NGram.compare('Ham', 'Spam', N=1)

0.4

* ngram similarity 계산 및 비슷한 아이템 찾기, 가장 비슷한 아이템 반환 가능

In [13]:
G = ngram.NGram(['joe','joseph','jon','john','sally', 'joen'])

In [20]:
G.search('jon')

[('jon', 1.0),
 ('joen', 0.375),
 ('john', 0.375),
 ('joe', 0.25),
 ('joseph', 0.18181818181818182)]

In [21]:
G.search('jo')

[('joe', 0.2857142857142857),
 ('jon', 0.2857142857142857),
 ('joen', 0.25),
 ('john', 0.25),
 ('joseph', 0.2)]

In [10]:
G.search('jon', threshold=0.3)

[('jon', 1.0), ('john', 0.375)]

In [18]:
G.find('jos')

'joseph'

# Comparing and searching strings - TF-IDF


* TF(단어 빈도수, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지 나타내는 값 DF(Document Frequency)는 특정 단어가 나타난 문서의 수를 말하며, 이 값의 역수를 IDF(inverse document frequence)라고 한다. 

* TF-IDF = TF*IDF = TF*log((N – n)/n)

* N : 전체 문서의 수
* n : 단어가 포함된 문서의 수 
IDF = log((N – n)/n) 


예를 들어서

* 전체 문서가 100,000,000 존재한다.
* gift 라는 단어가 DOC1에 2번, DOC2에 1번 그리고 존재하는 문서는 300,000
    - card 라는 단어가 DOC1에 3번, DOC2에 6번 그리고 존재하는 문서는 400,000
gift card 라는 단어를 입력했을 때 DOC1, DOC2 문서간의 유사도를 구해본다. 

먼저
DOC1 문서
gift  tf = 2 ,  IDF = log((100,000,000 - 300,000) / 300,000 ) = 5.0431
card  tf = 3 ,  IDF = log((100,000,000 - 400,000) / 400,000 ) = 7.1886

<img src = im3.png width = 600>

위와 같이 TF-IDF 결과값이 나왔으며 이를 좌표를 표현하면 아래와 같다.


<img src = im4.png width = 600>

* QUERY와 DOC1의 계산 
* DOC1·Query = 5.0431*2.5216 + 7.1886*2.3962 = 29.94200
* |DOC1|=(5.0431*5.0431 + 7.1886*7.1886) 1/2 = (25.4329 + 51.6760)1/2 = 8.78117
* |Query |=(2.5216*2.5216 + 2.3962*2.3962) 1/2 = (6.3585 + 5.7418) 1/2 = 3.47854
* Cosine angle=DOC1·Query/(|DOC1|*|Query |)=29.94200/(8.78117*3.47854)= 0.9802
                         

* QUERY와 DOC2의 계산 
* DOC2·Query = 2.5216*2.5216 + 14.3772*2.3962 = 40.8091
* |DOC2|=(2.5216*2.5216 + 14.3772*14.3772) 1/2 = (6.3585 + 206.7039) )1/2 = 14.5967
* |Query |=(2.5216*2.5216 + 2.3962*2.3962) 1/2 = (6.3585 + 5.7418) 1/2 = 3.47854
* Cosine angle=DOC2·Query/(|DOC2|*|Query |)=40.8091/(14.5967*3.47854) =0.80372

* DOC1 이 입력된 gift card 쿼리에 대해 DOC2보다 정확도가 높은 걸로 나온다. 


* 원 출처 http://www.miislita.com/information-retrieval-tutorial/cosine-similarity-tutorial.html
* TF-IDF에 대해서 알아보기 (http://codezip.tistory.com/429) 글 재인용 

# 2.2 Exploiting Linguistic Context for Keyword Extraction

<img src = im5.png width = 600>

* 작은 박스가 tokenization 이고 큰 박스가 chunking
* 이 안에는 정말 많은 candidate...eg., the yellow, yellow dog, the yellow dog....

* 정말 많은 candidate(토큰의 조합)중에서 많은 수가 광고에는 필요없는 정보들.
* 예를 들어 'Nikon unveils eight new Coolpix cameras'에서 광고를 위한 classification에 필요없는 noisy candidate는?

* 노이즈를 완화 하기 위한 방안들 : POS(Part of Speech) chunking - 자주 나타나는 패턴을 찾고 그에 따라서 태그를 붙임, 이를 통해 가능한 candidate를 줄임

# 2.3 Using External Resources

* 외부 정보를 통해 keyword extraction의 정확도를 높임
* 책에서 나오는 사례는 사전 지식을 이용한 상품 카테고리화를 통해 classification의 정확도 향상

# 2.4 Multi-label Learning with Millions of Labels

* Multi Label random forest